<span STYLE="font-size:150%"> 
    Segment polygonal regions from microCT scans
</span>

Docker image: gnasello/slicer-env:2023-10-10 \
Latest update: 11 October 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [ ]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
# this cell is tagged 'parameters'
volume_file = 'microCT_volume/microCT_volume.nrrd'
original_defect_path = 'segmented_volumes/originalDefect/'
file_segmentation = 'segmented_volumes/Bone.seg.nrrd'
directory_notebook = Path().parent.absolute()
sample_name = directory_notebook.stem

In [ ]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [ ]:
## mm
masterVolumeNode.GetSpacing()

Get ```originalDefect``` directory

In [ ]:
output_originalDefect_path = Path(original_defect_path)

Get norebook directory name

In [ ]:
sample_name

# Create segmentationNode

## Load Bone Segment as segmentationNode

In [ ]:
segmentationNode = slicer.util.loadSegmentation(str(Path(file_segmentation).resolve()), properties={"name": "Segmentation"})

Rename segment of the segmentation node 

In [ ]:
segmentation = segmentationNode.GetSegmentation()
segment = segmentation.GetNthSegment(0)
segment.SetName("Bone")

Create temporary segment editor to get access to effects

In [ ]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

## Load defect cylinders in segment node

Load `.stl` files as models to Slicer

In [ ]:
cylinder_files = [output_originalDefect_path / 'OriginalDefect.vtk']

color_list = [
    (77/255, 187/255, 213/255),
        ]

cylinderModel_nodes = []

for i, file in enumerate(cylinder_files):
    
    color = color_list[i]
    
    model = slicer.util.loadModel(file)
    model.SetName('Ingrowth_' + file.stem)
    cylinderModel_nodes.append(model)

    slicer.modules.segmentations.logic().ImportModelToSegmentationNode(model, segmentationNode)

    modelDisplayNode = model.GetDisplayNode()
    modelDisplayNode.SetColor(color[0], color[1], color[2])
    modelDisplayNode.SetOpacity(0.4)

Get closed surface representation of the segment, from [slicer scripting repository](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-nodes-warped-by-transform-sequence)

In [ ]:
segmentationNode.CreateClosedSurfaceRepresentation()

Set segment colors

In [ ]:
segments_color = {'Bone':(0.9450980392156862, 0.8392156862745098, 0.5686274509803921), # "Bone" color in Slicer
                  'Ingrowth_OriginalDefect':(77/255, 187/255, 213/255)
                  }

ps.segmentation.set_segments_color(segments_color, segmentationNode)

# Operation on segments

## Manual fix of the segmentation

Sometimes it might be necessary to remove speckles at the image boundaries. If so, use the `scissor` tool in the `Segment Editor` before proceeding with the rest of the script. 

## Intersect cylinder segments with bone segment

In [ ]:
ps.segmentation.logical_intersect('Ingrowth_OriginalDefect', 'Bone', segmentationNode, segmentEditorNode, segmentEditorWidget)

# Compute bone ingrowth volumes

Delete `Bone` segment before computing volumes

In [ ]:
segmentName = 'Bone'
segmentationNode.GetSegmentation().RemoveSegment(segmentName)

Compute segment statistics

In [ ]:
stats = ps.segmentation.segment_statistics(segmentationNode)

Store volume of each segment

In [ ]:
segment_names = []
volumes_ingrowth_mm3 = []

# Display volume of each segment
for segmentId in stats["SegmentIDs"]:
    
    volume_mm3 = stats[segmentId,"LabelmapSegmentStatisticsPlugin.volume_mm3"]
    volumes_ingrowth_mm3.append(volume_mm3)
    
    segmentName = segmentationNode.GetSegmentation().GetSegment(segmentId).GetName()
    segment_names.append(segmentName)

data_dict = {'segmentName': segment_names, 'volume_mm3':volumes_ingrowth_mm3}
df = pd.DataFrame(data_dict)
df

Add sample name columns to DataFrame

In [ ]:
df['sample'] = [sample_name] * len(df. index)
df

In [ ]:
outputfile = output_originalDefect_path / (sample_name + '_volume_ingrowth.csv')
df.to_csv(outputfile, index=False)

# Export segments

## Export all segments to individual `.stl` files

Get closed surface representation of the segment, from [slicer scripting repository](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-nodes-warped-by-transform-sequence)

In [ ]:
segmentationNode.CreateClosedSurfaceRepresentation()

In [ ]:
slicer.modules.segmentations.logic().ExportSegmentsClosedSurfaceRepresentationToFiles(str(output_originalDefect_path), segmentationNode)